* Install the oauth client lib by
: pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

* Enable the Oauth API key for Google Calendar API from the Devlopers API Console.
* A json file will be generated contaning the credentianls of the key for usage ("client_secret.json") save it same     directory as this script.
* The CSV file should contain the following columns :
    > A unique key value [Security value]
    > Date of the event [Result Date]
    > Description of the event
    > Any additional info that you might want to be added to your event like its repeat, reminder etc in different        columns
    


In [ ]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from dateutil.parser import parse
import numpy as np
import pandas as pd


In [ ]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)
credentials = flow.run_console()

In [ ]:
#to store the creds for next api calls.
pickle.dump(credentials, open("token.pkl", "wb")) 
credentials = pickle.load(open("token.pkl", "rb"))

In [ ]:
#initiating the service
service = build('calendar', 'v3', 
                credentials=credentials)

In [ ]:
#Sample for getting the next 10 events 
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(event['summary'],event['id'])


In [ ]:
# loading the csv 
df= pd.read_csv('Results.csv',sep=',')

In [ ]:
#for creating first 10 events in calendar
for i in df.head().index :
    event = {
  'summary': df['Security Name'][i],
  'description': 'XXXXXX',
  'start': {
    'dateTime': parse(df['Result Date'][i]).isoformat()+"Z",
    
  },
  'end': {
    'dateTime': parse(df['Result Date'][i]).isoformat()+"Z",
    'timeZone': 'America/Los_Angeles',
  },
}
    event = service.events().insert(calendarId='primary', body=event).execute()

In [ ]:
events = service.events().list(calendarId='primary', pageToken=page_token).execute() #df of events 
# events['items'] is a df with 'id' column from where you can get the event id by mathcing the unique 'summary' 
## that we added previously

#To delete the event you will need the event id
service.events().delete(calendarId='primary', eventId="XXXXXXX").execute()

In [ ]:
#for updating the events 

for i in df.head().index :
    EVENT = {
        'summary': df['Security Name'][i],
        'description': 'XXXXXX',
        'start':  {'dateTime': parse(df['Result Date'][i]).isoformat()+"Z"},
        'end':    {'dateTime': parse(df['Result Date'][i]).isoformat()+"Z"},

    }
    for event in events['items']:
        if(event['summary']==df['Security Name'][i]):
            service.events().update(calendarId='primary',eventId=event['id'], body=EVENT).execute()
    page_token = events.get('nextPageToken')
    if not page_token:
        continue

In [ ]:
# Will update other functionalities that can be done.